In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('E:/Master_thesis_REV/Datasets/Dataset_amazon_book_reviews/archive/books_data.csv')
ratings = pd.read_csv('E:/Master_thesis_REV/Datasets/Dataset_amazon_book_reviews/archive/Books_rating.csv')

In [3]:
data.head()

,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,['Religion'],NaN
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,['Fiction'],NaN
4,"Nation Dance: Religion, Identity and Cultural ...",NaN,['Edward Long'],NaN,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,2003-03-01,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,NaN


In [4]:
ratings.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
# from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import time
import psutil

# Load the dataset
df = pd.read_csv('E:/Master_thesis_REV/Datasets/Dataset_amazon_book_reviews/archive/Books_rating.csv')

# Display the first few rows
df.head()


,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


In [6]:
# columns available in the DataFrame
print("Available columns in the DataFrame:", df.columns)

Available columns in the DataFrame: Index(['Id', 'Title', 'Price', 'User_id', 'profileName', 'review/helpfulness',
       'review/score', 'review/time', 'review/summary', 'review/text'],
      dtype='object')


In [7]:
# Define a function to safely parse the helpfulness ratio
def parse_helpfulness_ratio(helpfulness):
    try:
        votes = list(map(int, helpfulness.split('/')))
        if len(votes) == 2 and votes[1] != 0:
            return votes[0] / votes[1]
    except:
        return np.nan
    return np.nan

# Calculate helpfulness ratio using the safe function
df['helpfulness_ratio'] = df['review/helpfulness'].apply(parse_helpfulness_ratio)

# Ensure all required columns are available before dropping NaNs
required_columns = ['review/score', 'Price', 'helpfulness_ratio', 'review/text']

# Check for missing values specifically in these columns
missing_data = df[required_columns].isnull().sum()
print("Missing values in each column before dropna:")
print(missing_data)

# Drop rows with missing values in these columns
df.dropna(subset=required_columns, inplace=True)

# Verify that the required columns have no missing values after cleaning
missing_data_after = df[required_columns].isnull().sum()
print("Missing values in each column after dropna:")
print(missing_data_after)

# Display the first few rows after cleaning
df.head()


Missing values in each column before dropna:
review/score               0
Price                2518829
helpfulness_ratio     885732
review/text                8
dtype: int64
Missing values in each column after dropna:
review/score         0
Price                0
helpfulness_ratio    0
review/text          0
dtype: int64


,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text,helpfulness_ratio
10,0829814000,Wonderful Worship in Smaller Churches,19.40,AZ0IOBU20TBOP,Rev. Pamela Tinnin,8/10,5.0,991440000,Outstanding Resource for Small Church Pastors,"I just finished the book, &quot;Wonderful Wors...",0.800000
11,0829814000,Wonderful Worship in Smaller Churches,19.40,A373VVEU6Z9M0N,Dr. Terry W. Dorsett,1/1,5.0,1291766400,Small Churches CAN Have Wonderful Worship,Many small churches feel like they can not hav...,1.000000
12,0829814000,Wonderful Worship in Smaller Churches,19.40,AGKGOH65VTRR4,"Cynthia L. Lajoy ""Cindy La Joy""",1/1,5.0,1248307200,Not Just for Pastors!,I just finished reading this amazing book and ...,1.000000
13,0829814000,Wonderful Worship in Smaller Churches,19.40,A3OQWLU31BU1Y,Maxwell Grant,1/1,5.0,1222560000,Small church pastor? This is the book on worship,I hadn't been a small church pastor very long ...,1.000000
14,0595344550,Whispers of the Wicked Saints,10.95,A3Q12RK71N74LB,Book Reader,7/11,1.0,1117065600,not good,I bought this book because I read some glowing...,0.636364


In [8]:
df.shape

(356273, 11)

In [9]:
# Fill NaN values in text columns with empty strings
df['review/text'].fillna('', inplace=True)
df['review/summary'].fillna('', inplace=True)

# Function for frequency encoding
def frequency_encoding(column):
    freq = column.value_counts() / len(column)
    return column.map(freq)

# Apply frequency encoding to the text columns
df['review_text_encoded'] = frequency_encoding(df['review/text'])
df['review_summary_encoded'] = frequency_encoding(df['review/summary'])

# Combine the encoded features with other numeric features
processed_df = pd.concat([df[['Price', 'helpfulness_ratio', 'review_text_encoded', 'review_summary_encoded']]], axis=1)

# Verify the shape of the processed DataFrame
print("Processed DataFrame shape:", processed_df.shape)
processed_df.head()

Processed DataFrame shape: (356273, 4)


,Price,helpfulness_ratio,review_text_encoded,review_summary_encoded
10,19.40,0.800000,0.000003,0.000003
11,19.40,1.000000,0.000003,0.000003
12,19.40,1.000000,0.000003,0.000003
13,19.40,1.000000,0.000003,0.000003
14,10.95,0.636364,0.000003,0.000031


In [10]:
# Define features (X) and target (y)
X = processed_df
y = df['review/score']

# Display the shapes of X and y
print(f'X shape: {X.shape}, y shape: {y.shape}')

X shape: (356273, 4), y shape: (356273,)


In [11]:
# Function to sample data (updated to handle full dataset properly)
def sample_data(X, y, sample_size):
    if isinstance(sample_size, float):
        if 0 < sample_size < 1.0:
            return train_test_split(X, y, test_size=0.2, train_size=sample_size, random_state=42)
        elif sample_size == 1.0:
            # Use the entire dataset for training, leave a small portion for testing
            return train_test_split(X, y, test_size=0.001, random_state=42)
        else:
            raise ValueError("sample_size as float must be in the range (0.0, 1.0) or equal to 1.0.")
    elif isinstance(sample_size, int):
        if sample_size > len(X):
            raise ValueError(f"sample_size {sample_size} exceeds the number of available samples {len(X)}.")
        sampled_X = X.sample(n=sample_size, random_state=42)
        sampled_y = y.loc[sampled_X.index]
        return train_test_split(sampled_X, sampled_y, test_size=0.2, random_state=42)
    else:
        raise ValueError("sample_size must be a float or an integer")


In [15]:
import xgboost as xgb

def calculate_metrics(X_train, X_test, y_train, y_test):
    model = xgb.XGBRegressor(objective='reg:squarederror', n_jobs=-1)

    # # Hyperparameter tuning
    # param_distributions = {
    #     'n_estimators': [50, 100, 200],
    #     'max_depth': [3, 4, 5, 6],
    #     'learning_rate': [0.01, 0.1, 0.2],
    #     'subsample': [0.6, 0.8, 1.0],
    #     'colsample_bytree': [0.6, 0.8, 1.0],
    #     'gamma': [0, 0.1, 0.2],
    # }

    param_distributions = {
    'n_estimators': [100, 300, 500],
    'max_depth': [3, 4, 5, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3],
    'min_child_weight': [1, 3, 5]
   }

    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='neg_mean_squared_error', random_state=42, verbose=2)

    start_time = time.time()
    start_cpu = psutil.cpu_percent(interval=None)
    random_search.fit(X_train, y_train)
    end_cpu = psutil.cpu_percent(interval=None)
    end_time = time.time()

    execution_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    y_pred = random_search.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    target_range = y_train.max() - y_train.min()
    nrmse = rmse / target_range
    
    memory_usage_MB = X_train.memory_usage(deep=True).sum() / (1024 ** 2)
    normalized_time = execution_time / memory_usage_MB
    
    return {
        'RMSE': rmse,
        'MAPE': mape,
        'R2': r2,
        'MSE': mse,
        'nRMSE': nrmse,
        'Execution Time (Raw)': execution_time,
        'Normalized Time (s/MB)': normalized_time,
        'Average CPU Usage': avg_cpu_usage
    }


In [16]:
# Define sample sizes
sample_sizes = [1.0, 0.5, 0.25, 0.125, 100, 1000, 10000, 100000]

# Initialize metrics storage
metrics_list = []
total_execution_time = 0
total_cpu_usage = 0
total_memory_usage_MB = 0

# Loop through each sample size
for size in sample_sizes:
    try:
        X_train_sample, X_test_sample, y_train_sample, y_test_sample = sample_data(X, y, size)
        metrics = calculate_metrics(X_train_sample, X_test_sample, y_train_sample, y_test_sample)
        metrics['Sample Size'] = size
        metrics_list.append(metrics)

        # Accumulate total metrics
        total_execution_time += metrics['Execution Time (Raw)']
        total_cpu_usage += metrics['Average CPU Usage']
        total_memory_usage_MB += X_train_sample.memory_usage(deep=True).sum() / (1024 ** 2)

        print(f"Metrics for sample size {size}:")
        for key, value in metrics.items():
            print(f"{key}: {value}")
        print("-" * 50)

    except Exception as e:
        print(f"An error occurred for sample size {size}: {e}")


Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] END colsample_bytree=0.6, gamma=0.2, learning_rate=0.01, max_depth=4, min_child_weight=5, n_estimators=300, subsample=1.0; total time=   2.1s
[CV] END colsample_bytree=0.6, gamma=0.2, learning_rate=0.01, max_depth=4, min_child_weight=5, n_estimators=300, subsample=1.0; total time=   1.6s
[CV] END colsample_bytree=0.6, gamma=0.2, learning_rate=0.01, max_depth=4, min_child_weight=5, n_estimators=300, subsample=1.0; total time=   1.5s
[CV] END colsample_bytree=0.6, gamma=0.2, learning_rate=0.01, max_depth=4, min_child_weight=5, n_estimators=300, subsample=1.0; total time=   1.5s
[CV] END colsample_bytree=0.6, gamma=0.2, learning_rate=0.01, max_depth=4, min_child_weight=5, n_estimators=300, subsample=1.0; total time=   2.0s
[CV] END colsample_bytree=1.0, gamma=0.1, learning_rate=0.01, max_depth=8, min_child_weight=5, n_estimators=100, subsample=0.8; total time=   1.5s
[CV] END colsample_bytree=1.0, gamma=0.1, learning_rate=

In [17]:
# Convert metrics to DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Calculate total metrics
total_avg_cpu_usage = total_cpu_usage / len(sample_sizes)
normalized_total_time = total_execution_time / total_memory_usage_MB

# Convert total execution time to minutes and seconds
total_minutes = int(total_execution_time // 60)
total_seconds = total_execution_time % 60

# Display total metrics
print(f"Total Execution Time for Entire Process (Raw): {total_minutes} minutes and {total_seconds:.2f} seconds")
print(f"Total Normalized Execution Time for Entire Process: {normalized_total_time:.8f} seconds per MB")
print(f"Total Average CPU Usage for Entire Process: {total_avg_cpu_usage:.2f}%")

# Display the metrics DataFrame
metrics_df


Total Execution Time for Entire Process (Raw): 21 minutes and 10.40 seconds
Total Normalized Execution Time for Entire Process: 44.02024954 seconds per MB
Total Average CPU Usage for Entire Process: 46.62%


,RMSE,MAPE,R2,MSE,nRMSE,Execution Time (Raw),Normalized Time (s/MB),Average CPU Usage,Sample Size
0,1.122365,0.359692,0.189086,1.259704,0.280591,339.394320,24.997523,38.55,1.000
1,1.165025,0.396475,0.189884,1.357284,0.291256,182.478919,26.853502,79.20,0.500
2,1.168291,0.401618,0.185335,1.364904,0.292073,100.024263,29.439036,79.80,0.250
3,1.168112,0.402260,0.185585,1.364486,0.292028,136.204612,80.175196,69.05,0.125
4,1.041355,0.242570,-0.191669,1.084419,0.260339,90.234299,29567.975078,19.90,100.000
5,1.271956,0.491598,0.113191,1.617872,0.317989,112.616440,3690.215492,29.15,1000.000
6,1.234751,0.448107,0.163000,1.524611,0.308688,147.443981,483.144438,25.40,10000.000
7,1.159321,0.396828,0.190611,1.344024,0.289830,162.004757,53.085719,31.90,100000.000


In [22]:
import xgboost as xgb

def calculate_metrics(X_train, X_test, y_train, y_test):
    model = xgb.XGBRegressor(objective='reg:squarederror', n_jobs=-1)

    # # Hyperparameter tuning
    # param_distributions = {
    #     'n_estimators': [50, 100, 200],
    #     'max_depth': [3, 4, 5, 6],
    #     'learning_rate': [0.01, 0.1, 0.2],
    #     'subsample': [0.6, 0.8, 1.0],
    #     'colsample_bytree': [0.6, 0.8, 1.0],
    #     'gamma': [0, 0.1, 0.2],
    # }

    param_distributions = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3],
    'min_child_weight': [1, 3, 5]
   }

    random_search = RandomizedSearchCV(model, param_distributions, n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42, verbose=2)

    start_time = time.time()
    start_cpu = psutil.cpu_percent(interval=None)
    random_search.fit(X_train, y_train)
    end_cpu = psutil.cpu_percent(interval=None)
    end_time = time.time()

    execution_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    y_pred = random_search.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    target_range = y_train.max() - y_train.min()
    nrmse = rmse / target_range
    
    memory_usage_MB = X_train.memory_usage(deep=True).sum() / (1024 ** 2)
    normalized_time = execution_time / memory_usage_MB
    
    return {
        'RMSE': rmse,
        'MAPE': mape,
        'R2': r2,
        'MSE': mse,
        'nRMSE': nrmse,
        'Execution Time (Raw)': execution_time,
        'Normalized Time (s/MB)': normalized_time,
        'Average CPU Usage': avg_cpu_usage
    }


In [23]:
# Define sample sizes
sample_sizes = [1.0, 0.5, 0.25, 0.125, 100, 1000, 10000, 100000]

# Initialize metrics storage
metrics_list = []
total_execution_time = 0
total_cpu_usage = 0
total_memory_usage_MB = 0

# Loop through each sample size
for size in sample_sizes:
    try:
        X_train_sample, X_test_sample, y_train_sample, y_test_sample = sample_data(X, y, size)
        metrics = calculate_metrics(X_train_sample, X_test_sample, y_train_sample, y_test_sample)
        metrics['Sample Size'] = size
        metrics_list.append(metrics)

        # Accumulate total metrics
        total_execution_time += metrics['Execution Time (Raw)']
        total_cpu_usage += metrics['Average CPU Usage']
        total_memory_usage_MB += X_train_sample.memory_usage(deep=True).sum() / (1024 ** 2)

        print(f"Metrics for sample size {size}:")
        for key, value in metrics.items():
            print(f"{key}: {value}")
        print("-" * 50)

    except Exception as e:
        print(f"An error occurred for sample size {size}: {e}")


Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END colsample_bytree=0.6, gamma=0.2, learning_rate=0.01, max_depth=4, min_child_weight=5, n_estimators=200, subsample=1.0; total time=   1.9s
[CV] END colsample_bytree=0.6, gamma=0.2, learning_rate=0.01, max_depth=4, min_child_weight=5, n_estimators=200, subsample=1.0; total time=   1.6s
[CV] END colsample_bytree=0.6, gamma=0.2, learning_rate=0.01, max_depth=4, min_child_weight=5, n_estimators=200, subsample=1.0; total time=   1.8s
[CV] END colsample_bytree=0.6, gamma=0.2, learning_rate=0.01, max_depth=4, min_child_weight=5, n_estimators=200, subsample=1.0; total time=   1.9s
[CV] END colsample_bytree=0.6, gamma=0.2, learning_rate=0.01, max_depth=4, min_child_weight=5, n_estimators=200, subsample=1.0; total time=   2.6s
[CV] END colsample_bytree=1.0, gamma=0.1, learning_rate=0.01, max_depth=8, min_child_weight=5, n_estimators=100, subsample=0.8; total time=   2.0s
[CV] END colsample_bytree=1.0, gamma=0.1, learning_rate=0

In [24]:
# Convert metrics to DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Calculate total metrics
total_avg_cpu_usage = total_cpu_usage / len(sample_sizes)
normalized_total_time = total_execution_time / total_memory_usage_MB

# Convert total execution time to minutes and seconds
total_minutes = int(total_execution_time // 60)
total_seconds = total_execution_time % 60

# Display total metrics
print(f"Total Execution Time for Entire Process (Raw): {total_minutes} minutes and {total_seconds:.2f} seconds")
print(f"Total Normalized Execution Time for Entire Process: {normalized_total_time:.8f} seconds per MB")
print(f"Total Average CPU Usage for Entire Process: {total_avg_cpu_usage:.2f}%")

# Display the metrics DataFrame
metrics_df


Total Execution Time for Entire Process (Raw): 6 minutes and 36.79 seconds
Total Normalized Execution Time for Entire Process: 13.74892681 seconds per MB
Total Average CPU Usage for Entire Process: 85.94%


,RMSE,MAPE,R2,MSE,nRMSE,Execution Time (Raw),Normalized Time (s/MB),Average CPU Usage,Sample Size
0,1.121710,0.359673,0.190033,1.258233,0.280427,149.418991,11.005207,60.25,1.000
1,1.164740,0.398158,0.190281,1.356618,0.291185,80.969998,11.915502,82.00,0.500
2,1.168030,0.402224,0.185700,1.364293,0.292007,49.113646,14.455077,78.95,0.250
3,1.170466,0.402453,0.182300,1.369990,0.292616,31.633146,18.620468,93.35,0.125
4,1.041355,0.242570,-0.191669,1.084419,0.260339,9.674926,3170.279766,94.70,100.000
5,1.277229,0.495065,0.105823,1.631315,0.319307,12.400761,406.348141,94.95,1000.000
6,1.243580,0.465429,0.150989,1.546490,0.310895,18.769122,61.502658,90.20,10000.000
7,1.159531,0.395977,0.190318,1.344512,0.289883,44.806313,14.682132,93.15,100000.000
